In [1]:
from models import build_model
from datasets import IndexedInputTargetTranslationDataset
from dictionaries import IndexDictionary
from losses import TokenCrossEntropyLoss, LabelSmoothingLoss
from metrics import AccuracyMetric
from optimizers import NoamOptimizer
from trainer import EpochSeq2SeqTrainer
from utils.log import get_logger
from utils.pipe import input_target_collate_fn

import torch
from torch.optim import Adam
from torch.utils.data import DataLoader
import numpy as np

from argparse import ArgumentParser
from datetime import datetime
import json
import random

In [2]:

with open('configs/example_config.json') as f:
    config = json.load(f)

for key, default_value in config.items():

    print(key, default_value, '\n')


dataset_limit None 

print_every 1 

save_every 1 

vocabulary_size None 

share_dictionary False 

positional_encoding True 

d_model 128 

layers_count 1 

heads_count 2 

d_ff 128 

dropout_prob 0.1 

label_smoothing 0.1 

optimizer Noam 

lr 0.001 

clip_grads True 

batch_size 10 

epochs 10 



In [14]:
!head -8 ./data/example/processed/raw-val.txt

Once upon a time there was a little girl who everyone liked a lot, but most of all her grandmother. She could give this child anything. Once she gave her a little red hat, which suited her so well that she never wanted to wear anything else. She was therefore called Little Red Riding Hood.	En gång för länge sedan fanns det en liten flicka, som alla tyckte mycket om, men mest av alla hennes mormor. Hon skulle kunna ge detta barn vad som helst. En gång gav hon henne en liten röd luva, som passade henne så bra att hon aldrig ville ha på sig något annat. Hon kallades därför för Rödluvan.
One day her mother said to her "The Little Red Riding Hood, here's a piece of freshly baked cake and a bottle of wine. Take them to your grandmother, she's a little sick and that would do her good. Go before it gets too hot. Go carefully and do not leave the road, because then you can fall and smash the bottle. When you enter her room, do not forget to say good morning, before you run around every corner."

In [12]:
!head -8 ./data/example/processed/vocabulary-target.txt

0	<PAD>	0
1	<UNK>	0
2	<StartSent>	0
3	<EndSent>	0
4	En	4
5	gång	4
6	för	15
7	länge	2


In [13]:
!head -8 ./data/example/processed/vocabulary-source.txt

0	<PAD>	0
1	<UNK>	0
2	<StartSent>	0
3	<EndSent>	0
4	Once	3
5	upon	1
6	a	31
7	time	1


In [3]:
source_dictionary = IndexDictionary.load('data/example/processed', mode='source', vocabulary_size=531)
target_dictionary = IndexDictionary.load('data/example/processed', mode='target', vocabulary_size=553)

In [4]:
train_dataset = IndexedInputTargetTranslationDataset(
        data_dir='data/example/processed',
        phase='train',
        vocabulary_size=531,
        limit=128)
i=0
batch=[]
for t_data in train_dataset:
    if i<8:
        print(type(t_data),type(t_data[0]),type(t_data[1]), len(t_data))
        print("source",t_data[0],'\n')
        print("inputs",t_data[1],'\n')
        print("targets",t_data[2],'\n')
        i+=1
        batch.append(t_data)
    else:break
len(batch)

<class 'tuple'> <class 'list'> <class 'list'> 3
source [4, 5, 6, 7, 8, 9, 6, 10, 11, 12, 13, 14, 6, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 4, 28, 29, 20, 6, 10, 30, 31, 32, 33, 20, 34, 35, 36, 28, 37, 38, 39, 40, 41, 42, 22, 9, 43, 44, 45, 46, 47, 48] 

inputs [2, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 15, 22, 23, 24, 25, 26, 27, 28, 29, 30, 14, 31, 4, 5, 32, 33, 34, 11, 12, 35, 36, 14, 37, 34, 38, 39, 40, 33, 41, 42, 43, 44, 45, 46, 47, 24, 48, 49, 6, 50] 

targets [4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 15, 22, 23, 24, 25, 26, 27, 28, 29, 30, 14, 31, 4, 5, 32, 33, 34, 11, 12, 35, 36, 14, 37, 34, 38, 39, 40, 33, 41, 42, 43, 44, 45, 46, 47, 24, 48, 49, 6, 50, 3] 

<class 'tuple'> <class 'list'> <class 'list'> 3
source [49, 50, 20, 51, 52, 39, 20, 53, 45, 46, 47, 54, 55, 6, 56, 18, 57, 58, 59, 60, 6, 61, 18, 62, 63, 64, 39, 65, 66, 67, 6, 10, 68, 60, 36, 69, 70, 20, 71, 72, 73, 74, 75, 76, 77, 72, 78, 60, 70, 79, 80, 81,

8

In [5]:
PAD_INDEX = 0
def input_target_collate_fn(batch):
    """merges a list of samples to form a mini-batch."""

    # indexed_sources = [sources for sources, inputs, targets in batch]
    # indexed_inputs = [inputs for sources, inputs, targets in batch]
    # indexed_targets = [targets for sources, inputs, targets in batch]

    sources_lengths = [len(sources) for sources, inputs, targets in batch]
    inputs_lengths = [len(inputs) for sources, inputs, targets in batch]
    targets_lengths = [len(targets) for sources, inputs, targets in batch]

    sources_max_length = max(sources_lengths)
    inputs_max_length = max(inputs_lengths)
    targets_max_length = max(targets_lengths)
    print(sources_max_length, inputs_max_length,targets_max_length)

    sources_padded = [sources + [PAD_INDEX] * (sources_max_length - len(sources)) for sources, inputs, targets in batch]
    inputs_padded = [inputs + [PAD_INDEX] * (inputs_max_length - len(inputs)) for sources, inputs, targets in batch]
    targets_padded = [targets + [PAD_INDEX] * (targets_max_length - len(targets)) for sources, inputs, targets in batch]

    sources_tensor = torch.tensor(sources_padded)
    inputs_tensor = torch.tensor(inputs_padded)
    targets_tensor = torch.tensor(targets_padded)

    # lengths = {
    #     'sources_lengths': torch.tensor(sources_lengths),
    #     'inputs_lengths': torch.tensor(inputs_lengths),
    #     'targets_lengths': torch.tensor(targets_lengths)
    # }

    return sources_tensor, inputs_tensor, targets_tensor

In [6]:


train_dataloader = DataLoader(
        train_dataset,
        batch_size=8,
        shuffle=True,
        collate_fn=input_target_collate_fn)

In [7]:
s,i,t=input_target_collate_fn(batch)
s.size(),i.size(),t.size()

80 81 81


(torch.Size([8, 80]), torch.Size([8, 81]), torch.Size([8, 81]))

In [8]:
!python train.py --data_dir=/workspace/parallel/out/HFout/ --save_config=checkpoints/example_config.json --positional_encoding --save_checkpoint=checkpoints/en2sv.pth --epochs 500 --save_log=logs/example.log --epochs 1500

[INFO] 2020-09-14 14:15:40 > Run name : d_model=128-layers_count=1-heads_count=2-pe=False-optimizer=Adam-2020_09_14_14_15_40
[INFO] 2020-09-14 14:15:40 > {'config': None, 'data_dir': 'data/example/processed', 'save_config': 'checkpoints/example_config.json', 'save_checkpoint': 'checkpoints/swedish.pth', 'save_log': 'logs/example.log', 'device': 'cuda', 'dataset_limit': None, 'print_every': 1, 'save_every': 1, 'vocabulary_size': None, 'positional_encoding': False, 'd_model': 128, 'layers_count': 1, 'heads_count': 2, 'd_ff': 128, 'dropout_prob': 0.1, 'label_smoothing': 0.1, 'optimizer': 'Adam', 'lr': 0.001, 'clip_grads': False, 'batch_size': 64, 'epochs': 1500}
[INFO] 2020-09-14 14:15:40 > Constructing dictionaries...
[INFO] 2020-09-14 14:15:40 > Source dictionary vocabulary : 531 tokens
[INFO] 2020-09-14 14:15:40 > Target dictionary vocabulary : 553 tokens
[INFO] 2020-09-14 14:15:40 > Building model...
[INFO] 2020-09-14 14:15:40 > Transformer(
  (encoder): TransformerEncoder(
    (embed

In [9]:
!python predict.py --source="But such an awful big mouth you have!" --config=checkpoints/example_config.json --checkpoint=checkpoints/swedish.pth

Constructing dictionaries...
Building model...
/workspace/beam.py:61: UserWarning: Integer division of tensors using div or / is deprecated, and in a future release div will perform true division as in Python 3. Use true_divide or floor_divide (// in Python) instead. (Triggered internally at  ../aten/src/ATen/native/BinaryOps.cpp:81.)
  prev_k = top_score_ids / vocabulary_size  # (beam_size, )
Candidate 0 : "Men en så förskräckligt stor mun
Candidate 1 : "Men en så förskräckligt stor mun du har!"
Candidate 2 : "Men en så förskräckligt stor mun mun du har!"
